## Coding Challenge 5: Web scrapping of an API: COMTRADE

In this notebook, I connect to the Comtrade API to download information of Chilean exports, disagreggated by product.

Based on: https://github.com/uncomtrade/comtradeapicall/blob/main/tests/example%20calling%20functions%20-%20notebook.ipynb

In [1]:
pip install -q comtradeapicall # Installing COMTRADE package to download their API data.

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install -q squarify

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install plotly


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import numpy as np
import altair as alt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import squarify
import requests
import comtradeapicall
import json
from urllib.parse import urlencode
import economic_complexity as ecplx
import time

In [5]:
subscription_key = 'f2f03f3dc19047f8a1d1e960fccb3ed1' # comtrade api subscription key (from comtradedeveloper.un.org)
directory = '<OUTPUT DIR>'  # output directory for downloaded files 
proxy_url = '<PROXY URL>'  # optional if you need proxy url

In [6]:
#set some variables
from datetime import date
from datetime import timedelta
today = date.today()
yesterday = today - timedelta(days=1)
lastweek = today - timedelta(days=7)

# Exports 2022 of the top and bottom 5 of ECI rank

Type of Product – Commodities (C) or Services (S)

Frequency – Annual (A) or Monthly (M) 

Classification Code – HS, SITC, BEC for Goods, EBOPS for Services

Period – YYYY for Annual or YYYYMM for Monthly

Reporter – All or Individual Reporters

Publication Date – Publication date range

aggregateBy - The level of disaggregation that you want in the products

### ECI Rankings

In [7]:
eci = pd.read_csv("https://raw.githubusercontent.com/IgnacioOrueta/ignacioorueta.github.io/refs/heads/main/Project/Section1/data/map_data.csv")

eci = eci[eci['year']==2022]

eci = eci.filter(['country', 'ISO3','hs_eci_rank'])

eci.sort_values(by='hs_eci_rank',inplace=True)

top5    = eci[:5]
bottom5 = eci[-5:]

iso3_top5_list = eci['ISO3'][:5].tolist()
iso3_bottom5_list = eci['ISO3'][-6:].tolist() # I will take 6 since CHAD does not have data

iso3_top5 = ','.join(iso3_top5_list)
iso3_bottom5 = ','.join(iso3_bottom5_list)

country_code_top5 = comtradeapicall.convertCountryIso3ToCode(iso3_top5) # Switzerland and Germany have 2 codes
country_code_bottom5 = comtradeapicall.convertCountryIso3ToCode(iso3_bottom5)

top5_code = country_code_top5.split(',')
bottom5_code = country_code_bottom5.split(',')

top5_name = ['DEU', 'DEU', 'JPN', 'KOR', 'SGP', 'CHE', 'CHE']
bottom5_name = ['MOZ','TCD', 'COG', 'COD', 'NGA', 'PNG']

products_names = pd.read_csv("../data/umap_product_space_hs92.csv") # Product Space coordinates


### API call

In [8]:
#Here, I call the API for 5 most complex countries

# Initialize an empty list to store the dataframes
dataframes = []

for i in range(0,len(top5_name)):
    country_name = top5_name[i] 
    country_code = top5_code[i]
    exp_raw = comtradeapicall.getFinalData(subscription_key=subscription_key, typeCode='C', freqCode='A', clCode='HS', period='2022',
                                        reporterCode=country_code, cmdCode=None, flowCode='X', partnerCode='0', #152 is Chile's country code
                                        partner2Code=None,
                                        customsCode=None, motCode=None, maxRecords=None, format_output='JSON',
                                        aggregateBy=None, breakdownMode='classic', countOnly=None, includeDesc=True)
    
    # Convert the JSON response to a dataframe and append it to the list
    df = pd.DataFrame(exp_raw)
    dataframes.append(df)
    
    # Assign the result to a dynamically named variable
    globals()[f'exp_raw_{country_name}'] = df
    
    # Sleep for 2 seconds between each loop iteration
    time.sleep(2)
    print(f"{country_name} downloaded")

# Concatenate all dataframes in the list into a single dataframe
top5_raw = pd.concat(dataframes, ignore_index=True)

# Print the concatenated dataframe
top5_raw.head()


DEU downloaded
DEU downloaded
JPN downloaded
KOR downloaded
SGP downloaded
CHE downloaded
CHE downloaded


C:\Users\iorueta\AppData\Local\Temp\ipykernel_1984\949990602.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  top5_raw = pd.concat(dataframes, ignore_index=True)


,typeCode,freqCode,refPeriodId,refYear,refMonth,period,reporterCode,reporterISO,reporterDesc,flowCode,...,netWgt,isNetWgtEstimated,grossWgt,isGrossWgtEstimated,cifvalue,fobvalue,primaryValue,legacyEstimationFlag,isReported,isAggregate
0,C,A,20220101,2022,52,2022,276,DEU,Germany,X,...,NaN,False,0.0,False,NaN,1.448209e+09,1.448209e+09,0,False,True
1,C,A,20220101,2022,52,2022,276,DEU,Germany,X,...,NaN,False,0.0,False,NaN,2.811280e+08,2.811280e+08,0,False,True
2,C,A,20220101,2022,52,2022,276,DEU,Germany,X,...,389234.105,True,0.0,False,NaN,3.267086e+07,3.267086e+07,6,False,True
3,C,A,20220101,2022,52,2022,276,DEU,Germany,X,...,1702212.418,True,0.0,False,NaN,2.484569e+08,2.484569e+08,4,False,True
4,C,A,20220101,2022,52,2022,276,DEU,Germany,X,...,NaN,False,0.0,False,NaN,2.464130e+02,2.464130e+02,0,False,True


In [9]:
#Here, I call the API for 5 most complex countries

# Initialize an empty list to store the dataframes
dataframes = []

for i in range(0,len(bottom5_name)):
    country_name = bottom5_name[i] 
    country_code = bottom5_code[i]
    
    if country_name != 'PNG':
        exp_raw = comtradeapicall.getFinalData(subscription_key=subscription_key, typeCode='C', freqCode='A', clCode='HS', period='2022',
                                        reporterCode=country_code, cmdCode=None, flowCode='X', partnerCode='0', #152 is Chile's country code
                                        partner2Code=None,
                                        customsCode=None, motCode=None, maxRecords=None, format_output='JSON',
                                        aggregateBy=None, breakdownMode='classic', countOnly=None, includeDesc=True)

    if country_name == 'PNG':
# Last data for Papua New guinea is 2020. I will make call again an append to the bottom5_raw
        exp_raw = comtradeapicall.getFinalData(subscription_key=subscription_key, typeCode='C', freqCode='A', clCode='HS', period='2020',
                                        reporterCode='598', cmdCode=None, flowCode='X', partnerCode='0', #152 is Chile's country code
                                        partner2Code=None,
                                        customsCode=None, motCode=None, maxRecords=None, format_output='JSON',
                                        aggregateBy=None, breakdownMode='classic', countOnly=None, includeDesc=True)

    # Convert the JSON response to a dataframe and append it to the list
    df = pd.DataFrame(exp_raw)
    dataframes.append(df)
    
    # Assign the result to a dynamically named variable
    globals()[f'exp_raw_{country_name}'] = df
    
    # Sleep for 2 seconds between each loop iteration
    time.sleep(2)
    print(f"{country_name} downloaded")

# Concatenate all dataframes in the list into a single dataframe
bottom5_raw = pd.concat(dataframes, ignore_index=True)

# Print the concatenated dataframe
bottom5_raw.head()

MOZ downloaded
TCD downloaded
COG downloaded
COD downloaded
NGA downloaded
PNG downloaded


,typeCode,freqCode,refPeriodId,refYear,refMonth,period,reporterCode,reporterISO,reporterDesc,flowCode,...,netWgt,isNetWgtEstimated,grossWgt,isGrossWgtEstimated,cifvalue,fobvalue,primaryValue,legacyEstimationFlag,isReported,isAggregate
0,C,A,20220101,2022,52,2022,178,COG,Congo,X,...,32970.00,False,0.0,False,None,66779.781,66779.781,0,False,True
1,C,A,20220101,2022,52,2022,178,COG,Congo,X,...,1224.83,False,0.0,False,None,9735.726,9735.726,0,False,True
2,C,A,20220101,2022,52,2022,178,COG,Congo,X,...,350.00,False,0.0,False,None,2197.413,2197.413,0,False,True
3,C,A,20220101,2022,52,2022,178,COG,Congo,X,...,173995.00,False,0.0,False,None,109734.239,109734.239,2,False,True
4,C,A,20220101,2022,52,2022,178,COG,Congo,X,...,158.00,False,0.0,False,None,126.251,126.251,0,False,True


In [10]:
top5_raw['reporterDesc'].value_counts()

reporterDesc
Germany          6683
Switzerland      6353
Singapore        6239
Rep. of Korea    6235
Japan            6122
Name: count, dtype: int64

In [11]:
bottom5_raw['reporterDesc'].value_counts()

reporterDesc
Mozambique                2816
Congo                     1725
Papua New Guinea          1709
Dem. Rep. of the Congo    1433
Nigeria                    972
Name: count, dtype: int64

### Merging downloaded data

### Cleaning and filtering process

In [ ]:
top5 = top5_raw[['period', 'reporterDesc', 'cmdCode','cmdDesc', 'aggrLevel', 'fobvalue', 'primaryValue']]
bottom5 = bottom5_raw[['period', 'reporterDesc', 'cmdCode','cmdDesc', 'aggrLevel', 'fobvalue', 'primaryValue']]

# Extract first 2 digits of product code to aggregate them after
# HS4
top5['HS4'] = top5['cmdCode'].astype(str).str[:4]
bottom5['HS4'] = bottom5['cmdCode'].astype(str).str[:4]

# Keeping only HS4
top5 = top5[top5['aggrLevel']==4]
bottom5 = bottom5[bottom5['aggrLevel']==4]

top5 = top5[top5['HS4'] != 'TOTA'] # Dropping total
bottom5 = bottom5[bottom5['HS4'] != 'TOTA'] # Dropping total


C:\Users\iorueta\AppData\Local\Temp\ipykernel_1984\3127494182.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top5['HS4'] = top5['cmdCode'].astype(str).str[:4]
C:\Users\iorueta\AppData\Local\Temp\ipykernel_1984\3127494182.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bottom5['HS4'] = bottom5['cmdCode'].astype(str).str[:4]


In [ ]:
top5_hs4 = top5.groupby(['HS4', 'reporterDesc']).agg({
    'fobvalue': 'sum',
    'period': 'first'
}).reset_index()

top5_hs4.head()

bottom5_hs4 = bottom5.groupby(['HS4', 'reporterDesc']).agg({
    'fobvalue': 'sum',
    'period': 'first'
}).reset_index()

# Extract first 2 digits for HS2
top5_hs4['HS2'] = top5_hs4['HS4'].astype(str).str[:2]
bottom5_hs4['HS2'] = bottom5_hs4['HS4'].astype(str).str[:2]


bottom5_hs4.head()

,HS4,reporterDesc,fobvalue,period,HS2
0,0101,Dem. Rep. of the Congo,4450.660,2022,01
1,0102,Dem. Rep. of the Congo,896.059,2022,01
2,0103,Dem. Rep. of the Congo,1740.227,2022,01
3,0104,Dem. Rep. of the Congo,39.736,2022,01
4,0105,Congo,7191.533,2022,01


In [ ]:
# In share of total:
top5_hs4['percent_fobvalue'] = top5_hs4['fobvalue'] / top5_hs4['fobvalue'].sum() * 100
top5_hs4 = top5_hs4.reset_index()

bottom5_hs4['percent_fobvalue'] = bottom5_hs4['fobvalue'] / bottom5_hs4['fobvalue'].sum() * 100
exp_bottom5_hs4 = bottom5_hs4.reset_index()
exp_bottom5_hs4

,index,HS4,reporterDesc,fobvalue,period,HS2,percent_fobvalue
0,0,0101,Dem. Rep. of the Congo,4450.660,2022,01,4.174122e-06
1,1,0102,Dem. Rep. of the Congo,896.059,2022,01,8.403831e-07
2,2,0103,Dem. Rep. of the Congo,1740.227,2022,01,1.632099e-06
3,3,0104,Dem. Rep. of the Congo,39.736,2022,01,3.726704e-08
4,4,0105,Congo,7191.533,2022,01,6.744693e-06
...,...,...,...,...,...,...,...
2547,2547,9703,Mozambique,50360.620,2022,97,4.723150e-05
2548,2548,9705,Mozambique,46807.230,2022,97,4.389890e-05
2549,2549,9705,Papua New Guinea,6.722,2020,97,6.304334e-09
2550,2550,9999,Congo,1278654.800,2022,99,1.199207e-03


In [15]:
# Merge with cluster names
products_names.rename(columns={
    'product_code': 'HS4',
}, inplace=True)

products_names = products_names[['HS4', 'cluster_name']]

products_names['HS4'] = products_names['HS4'].astype(str)

# Merge with HS2 codes
hs2 = pd.read_excel('../data/HSCodeandDescription.xlsx')
hs2 = hs2[hs2['Level'] == 2]

hs2.rename(columns={
    'Code': 'HS2',
    'Description': 'description_hs2'
}, inplace=True)

hs2 = hs2[['HS2', 'description_hs2']]

# Top5
top5_hs_merged = pd.merge(top5_hs4, hs2, on='HS2', how='left')
top5_hs_merged = pd.merge(top5_hs_merged, products_names, on='HS4', how='left')

# Bottom5
bottom5_hs_merged = pd.merge(bottom5_hs4, hs2, on='HS2', how='left')
bottom5_hs_merged = pd.merge(bottom5_hs_merged, products_names, on='HS4', how='left')

# Split the 'Contains' column by commas and expand it into separate columns
top5_hs_merged['desc_hs2']= top5_hs_merged['description_hs2'].str.split(';', expand=False).str[0]
bottom5_hs_merged['desc_hs2']= bottom5_hs_merged['description_hs2'].str.split(';', expand=False).str[0]

# Applying a lambda function to truncate strings. First 150 characters only
top5_hs_merged['desc_hs2'] = top5_hs_merged['desc_hs2'].astype(str).apply(lambda x: x[:150] + '...' if len(x) > 100 else x)
bottom5_hs_merged['desc_hs2'] = bottom5_hs_merged['desc_hs2'].astype(str).apply(lambda x: x[:150] + '...' if len(x) > 100 else x)

# Leading with NAs
top5_hs_merged['HS4'] = top5_hs_merged['HS4'].astype(int)
top5_hs_merged['cluster_name'] = np.where(top5_hs_merged['HS4'] < 2600, top5_hs_merged['cluster_name'].fillna('Agriculture'), top5_hs_merged['cluster_name']) 
top5_hs_merged['cluster_name'] = np.where((top5_hs_merged['HS4'] > 2600) & (top5_hs_merged['HS4'] < 2700), top5_hs_merged['cluster_name'].fillna('Minerals'), top5_hs_merged['cluster_name']) 
top5_hs_merged['cluster_name'] = np.where((top5_hs_merged['HS4'] > 2700) & (top5_hs_merged['HS4'] < 3105), top5_hs_merged['cluster_name'].fillna('Industrial Chemicals and Metals'), top5_hs_merged['cluster_name']) 
top5_hs_merged['cluster_name'] = np.where((top5_hs_merged['HS4'] > 3105) & (top5_hs_merged['HS4'] < 4100), top5_hs_merged['cluster_name'].fillna('Construction, Building, and Home Supplies'), top5_hs_merged['cluster_name']) 
top5_hs_merged['cluster_name'] = np.where((top5_hs_merged['HS4'] > 4100) & (top5_hs_merged['HS4'] < 6006), top5_hs_merged['cluster_name'].fillna('Textile and Home Goods'), top5_hs_merged['cluster_name']) 
top5_hs_merged['cluster_name'] = np.where((top5_hs_merged['HS4'] > 6006) & (top5_hs_merged['HS4'] < 6703), top5_hs_merged['cluster_name'].fillna('Textile Apparel and Accessories'), top5_hs_merged['cluster_name']) 
top5_hs_merged['cluster_name'] = np.where((top5_hs_merged['HS4'] > 6006) & (top5_hs_merged['HS4'] < 6703), top5_hs_merged['cluster_name'].fillna('Metalworking and Electrical Machinery and Parts'), top5_hs_merged['cluster_name']) 

# Assign the correct cluster names in these cases of desc_hs2 
top5_hs_merged['cluster_name'] = np.where(top5_hs_merged['desc_hs2'] == 'Pharmaceutical products', 'Industrial Chemicals and Metals', top5_hs_merged['cluster_name'])
top5_hs_merged['cluster_name'] = np.where(top5_hs_merged['desc_hs2'] == 'Copper and articles thereof', 'Minerals', top5_hs_merged['cluster_name'])

bottom5_hs_merged['HS4'] = bottom5_hs_merged['HS4'].astype(int)
bottom5_hs_merged['cluster_name'] = np.where(bottom5_hs_merged['HS4'] < 2600, bottom5_hs_merged['cluster_name'].fillna('Agriculture'), bottom5_hs_merged['cluster_name']) 
bottom5_hs_merged['cluster_name'] = np.where((bottom5_hs_merged['HS4'] > 2600) & (bottom5_hs_merged['HS4'] < 2700), bottom5_hs_merged['cluster_name'].fillna('Minerals'), bottom5_hs_merged['cluster_name']) 
bottom5_hs_merged['cluster_name'] = np.where((bottom5_hs_merged['HS4'] > 2700) & (bottom5_hs_merged['HS4'] < 3105), bottom5_hs_merged['cluster_name'].fillna('Industrial Chemicals and Metals'), bottom5_hs_merged['cluster_name']) 
bottom5_hs_merged['cluster_name'] = np.where((bottom5_hs_merged['HS4'] > 3105) & (bottom5_hs_merged['HS4'] < 4100), bottom5_hs_merged['cluster_name'].fillna('Construction, Building, and Home Supplies'), bottom5_hs_merged['cluster_name']) 
bottom5_hs_merged['cluster_name'] = np.where((bottom5_hs_merged['HS4'] > 4100) & (bottom5_hs_merged['HS4'] < 6006), bottom5_hs_merged['cluster_name'].fillna('Textile and Home Goods'), bottom5_hs_merged['cluster_name']) 
bottom5_hs_merged['cluster_name'] = np.where((bottom5_hs_merged['HS4'] > 6006) & (bottom5_hs_merged['HS4'] < 6703), bottom5_hs_merged['cluster_name'].fillna('Textile Apparel and Accessories'), bottom5_hs_merged['cluster_name']) 
bottom5_hs_merged['cluster_name'] = np.where((bottom5_hs_merged['HS4'] > 6006) & (bottom5_hs_merged['HS4'] < 6703), bottom5_hs_merged['cluster_name'].fillna('Metalworking and Electrical Machinery and Parts'), bottom5_hs_merged['cluster_name']) 

# Assign the correct cluster names in these cases of desc_hs2 
bottom5_hs_merged['cluster_name'] = np.where(bottom5_hs_merged['desc_hs2'] == 'Pharmaceutical products', 'Industrial Chemicals and Metals', bottom5_hs_merged['cluster_name'])
bottom5_hs_merged['cluster_name'] = np.where(bottom5_hs_merged['desc_hs2'] == 'Copper and articles thereof', 'Minerals', bottom5_hs_merged['cluster_name'])

# I cannot distinguish categories far from that. Thus, I will simply drop them to avoid mistakes.
top5_hs_merged.dropna(subset='cluster_name', inplace=True)
bottom5_hs_merged.dropna(subset='cluster_name', inplace=True)


In [16]:
# Creating country var with the year of the data: 
top5_hs_merged['country'] = top5_hs_merged['reporterDesc'] + ' - ' + top5_hs_merged['period']
bottom5_hs_merged['country'] = bottom5_hs_merged['reporterDesc'] + ' - ' + bottom5_hs_merged['period']

### Saving CSVs

In [17]:
top5_hs_merged.to_csv('../data/top5_countries.csv', index=False)  
bottom5_hs_merged.to_csv('../data/bottom5_countries.csv', index=False)  

### Plotting

#### a) Top 5

In [20]:
# Plotting a tree map 
 # See this very useful example: https://stackoverflow.com/questions/64100889/add-dropdown-menu-to-plotly-express-treemap

# Load the data
data_url = "https://raw.githubusercontent.com/IgnacioOrueta/ignacioorueta.github.io/refs/heads/main/Project/Section3/data/top5_countries.csv"
data = pd.read_csv(data_url)

# Define custom colors for each cluster_name
custom_colors = {
        'Agriculture': '#4E79A7',
        'Industrial Chemicals and Metals': '#76B7B2',
        'Textile and Home Goods': '#FF9DA7',
        'Metalworking and Electrical Machinery and Parts': '#59A14F',
        'Construction, Building, and Home Supplies': '#F28E2B',
        'Electronic and Electrical Goods': '#E15759',
        'Textile Apparel and Accessories': '#AF7AA1',
        'Minerals': '#EDC949',     
}

# Tableau colors
#4E79A7 (Blue)
#F28E2B (Orange)
#E15759 (Red)
# #76B7B2 (Teal)
# #59A14F (Green)
# #EDC949 (Yellow)
# #AF7AA1 (Purple)
#FF9DA7 (Pink)

dfs = list(data.groupby("country",))

first_title = dfs[0][0]
traces = []
buttons = []
for i,d in enumerate(dfs):
        visible = [False] * len(dfs)
        visible[i] = True
        name = d[0]
        traces.append(
            px.treemap(d[1],
                    path=['cluster_name', 'desc_hs2'],
                    values='percent_fobvalue',
                    color='cluster_name',
                    color_discrete_map=custom_colors).update_traces(visible=True if i==0 else False).data[0]
        )
        
        buttons.append(dict(label=name,
                        method="update",
                        args=[{"visible":visible}]))  # Creating the dropdown menu

updatemenus = [{
        'active': 0,
        'buttons': buttons,
        'x': 0.12,  # Adjust the x position
        'y': 1.15,  # Adjust the y position
        'xanchor': 'right',  # Anchor the dropdown to the right
        'yanchor': 'top'  # Anchor the dropdown to the top
    }]

fig = go.Figure(data=traces,
                    layout=dict(updatemenus=updatemenus))

# Add title and subtitle
fig.update_layout(
    title={
        'text': "Exports matrix of the 5 most complex countries",
        'y':0.98,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
            'font': {'size': 23, 'color': 'black'}
        },
    annotations=[
        dict(
            text="Share of total value exported (FOB)",
            x=0.5,
            y=1.18,
            xref="paper",
            yref="paper",
            showarrow=False,
            font=dict(size=17, color='black')
            ),
        dict(
            text="Source: Harvard Atlas of Economic Complexity and COMTRADE (2022).",
            x=0.5,
            y=1.12,
            xref="paper",
            yref="paper",
            showarrow=False,
            font=dict(size=12, color='black')
        )
        ],
    margin=dict(t=100, l=25, r=25, b=0),
    showlegend=True,
    )

# Show percentages in each quadrant
fig.update_traces(textinfo='label+percent entry', textfont_size=15)

# Create white borders between categories
fig.update_traces(marker=dict(line=dict(width=1)))

# Remove all tooltips except the label and category
fig.update_traces(hovertemplate='%{label}<br>%{value:.1f}%<br>Category: %{customdata[0]}<extra></extra>')

# Erase the border of the treemap graph
fig.update_layout(xaxis_showgrid=False, yaxis_showgrid=False)

# Remove the outer border of the chart
fig.update_layout(
        paper_bgcolor='white',
        plot_bgcolor='white',
        margin=dict(t=100, l=25, r=25, b=0)
)
    
pio.write_json(fig, '../figures/figure3.json', pretty=True) # JSON
pio.write_html(fig, '../figures/figure3.html') #HTML 

# Show the figure
fig.show()

#### Bottom 5

In [21]:
# Plotting a tree map 
 # See this very useful example: https://stackoverflow.com/questions/64100889/add-dropdown-menu-to-plotly-express-treemap

# Load the data
data_url = "https://raw.githubusercontent.com/IgnacioOrueta/ignacioorueta.github.io/refs/heads/main/Project/Section3/data/bottom5_countries.csv"
data = pd.read_csv(data_url)

# Define custom colors for each cluster_name
custom_colors = {
        'Agriculture': '#4E79A7',
        'Industrial Chemicals and Metals': '#76B7B2',
        'Textile and Home Goods': '#FF9DA7',
        'Metalworking and Electrical Machinery and Parts': '#59A14F',
        'Construction, Building, and Home Supplies': '#F28E2B',
        'Electronic and Electrical Goods': '#E15759',
        'Textile Apparel and Accessories': '#AF7AA1',
        'Minerals': '#EDC949',     
}

# Tableau colors
#4E79A7 (Blue)
#F28E2B (Orange)
#E15759 (Red)
# #76B7B2 (Teal)
# #59A14F (Green)
# #EDC949 (Yellow)
# #AF7AA1 (Purple)
#FF9DA7 (Pink)

dfs = list(data.groupby("country",))

first_title = dfs[0][0]
traces = []
buttons = []
for i,d in enumerate(dfs):
        visible = [False] * len(dfs)
        visible[i] = True
        name = d[0]
        traces.append(
            px.treemap(d[1],
                    path=['cluster_name', 'desc_hs2'],
                    values='percent_fobvalue',
                    color='cluster_name',
                    color_discrete_map=custom_colors).update_traces(visible=True if i==0 else False).data[0]
        )
        
        buttons.append(dict(label=name,
                        method="update",
                        args=[{"visible":visible}]))  # Creating the dropdown menu

updatemenus = [{
        'active': 0,
        'buttons': buttons,
        'x': 0.16,  # Adjust the x position
        'y': 1.15,  # Adjust the y position
        'xanchor': 'right',  # Anchor the dropdown to the right
        'yanchor': 'top'  # Anchor the dropdown to the top
    }]

fig = go.Figure(data=traces,
                    layout=dict(updatemenus=updatemenus))

# Add title and subtitle
fig.update_layout(
    title={
        'text': "Exports matrix of the 5 least complex countries",
        'y':0.98,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
            'font': {'size': 23, 'color': 'black' }
        },
    annotations=[
        dict(
            text="Share of total value exported (FOB)",
            x=0.5,
            y=1.18,
            xref="paper",
            yref="paper",
            showarrow=False,
            font=dict(size=17, color='black')
            ),
        dict(
            text="Source: Harvard Atlas of Economic Complexity and COMTRADE (2022).",
            x=0.5,
            y=1.12,
            xref="paper",
            yref="paper",
            showarrow=False,
            font=dict(size=12, color='black')
        )
        ],
    margin=dict(t=100, l=25, r=25, b=0),
    showlegend=True,
    )

# Show percentages in each quadrant
fig.update_traces(textinfo='label+percent entry', textfont_size=15)

# Create white borders between categories
fig.update_traces(marker=dict(line=dict(width=1)))

# Remove all tooltips except the label and category
fig.update_traces(hovertemplate='%{label}<br>%{value:.1f}%<br>Category: %{customdata[0]}<extra></extra>')

# Erase the border of the treemap graph
fig.update_layout(xaxis_showgrid=False, yaxis_showgrid=False)

# Remove the outer border of the chart
fig.update_layout(
        paper_bgcolor='white',
        plot_bgcolor='white',
        margin=dict(t=100, l=25, r=25, b=0)
)
    
pio.write_json(fig, '../figures/figure3b.json', pretty=True) # JSON
pio.write_html(fig, '../figures/figure3b.html') #HTML 

# Show the figure
fig.show()